# **CAP 5768: Introduction to Data Science**
## Final Project Program: EDA's

In [56]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import os

In [57]:
data = pd.read_csv(r'filtered_pollution_us_2000_2016.csv')
pd.set_option('display.float_format', '{:.2f}'.format)

In [58]:
# Display first few rows
data.head()

,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,19.04,49.00,19,46,0.02,0.04,10,34,3.00,9.00,21,13.00,1.15,4.20,21,NaN
1,19.04,49.00,19,46,0.02,0.04,10,34,3.00,9.00,21,13.00,0.88,2.20,23,25.00
2,19.04,49.00,19,46,0.02,0.04,10,34,2.98,6.60,23,NaN,1.15,4.20,21,NaN
3,19.04,49.00,19,46,0.02,0.04,10,34,2.98,6.60,23,NaN,0.88,2.20,23,25.00
4,22.96,36.00,19,34,0.01,0.03,10,27,1.96,3.00,22,4.00,0.85,1.60,23,NaN


In [59]:
# Display columns
print(f"Features from deafult dataset:\n {data.columns}")

Features from deafult dataset:
 Index(['NO2 Mean', 'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI',
       'O3 Mean', 'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', 'SO2 Mean',
       'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI', 'CO Mean',
       'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI'],
      dtype='object')


In [60]:
# Check for missing values
print(f"Missing values in each feature: \n{data.isnull().sum()}")

# Checking for duplicate rows
print(f"\nNumber of duplicate rows in dataset: \n{data.duplicated().sum()}")

# Dropping any duplicate rows
data.drop_duplicates(inplace= True)

Missing values in each feature: 
NO2 Mean                  0
NO2 1st Max Value         0
NO2 1st Max Hour          0
NO2 AQI                   0
O3 Mean                   0
O3 1st Max Value          0
O3 1st Max Hour           0
O3 AQI                    0
SO2 Mean                  0
SO2 1st Max Value         0
SO2 1st Max Hour          0
SO2 AQI              872907
CO Mean                   0
CO 1st Max Value          0
CO 1st Max Hour           0
CO AQI               873323
dtype: int64

Number of duplicate rows in dataset: 
5110


In [61]:
# Before checking threshold for features
number_rows, number_columns = data.shape
print(f"Raw Data: {number_rows} samples and {number_columns} features")


# Removing Any Feature with >= 80% Missing Values
threshold = 0.8
retain_columns = [col for col in data.columns if data[col].isnull().mean() < threshold]
data = data[retain_columns]

print(f"Features Retained After Removing >= 80% Missing Values: {len(retain_columns)}")

Raw Data: 1741551 samples and 16 features
Features Retained After Removing >= 80% Missing Values: 16


In [62]:
# Solving the NaN values for SO2 AQI

# Split Data where there are NaN and Non-NaN values
features = ["SO2 Mean", "SO2 1st Max Value", "SO2 1st Max Hour"]
so2_known = data.dropna(subset=["SO2 AQI"])
so2_unknown = data[data["SO2 AQI"].isna()]

# Using Linear Regression to fill in where NaN exists
so2_model = LinearRegression().fit(so2_known[features], so2_known["SO2 AQI"])
data.loc[data["SO2 AQI"].isna(), "SO2 AQI"] = so2_model.predict(so2_unknown[features])

In [63]:
# Solving the NaN values for CO AQI

# Split Data where there are NaN and Non-NaN values
features = ["CO Mean", "CO 1st Max Value", "CO 1st Max Hour"]
co_known = data.dropna(subset=["CO AQI"])
co_unknown = data[data["CO AQI"].isna()]

# Using Linear Regression to fill in where NaN exists
co_model = LinearRegression().fit(co_known[features], co_known["CO AQI"])
data.loc[data["CO AQI"].isna(), "CO AQI"] = co_model.predict(co_unknown[features])

# Handling of NaN values
* We identified that the CO AQI and SO2 AQI features were the only ones containing NaN values. Research indicated that this was an intentional practice by the centers recording these measurements.
* Consequently, we observed that removing any sample with these NaN values would have resulted in a loss of over 75% of our dataset.
* Therefore, we chose to retain these samples and imputed the missing values by performing linear regression on these features. This allowed us to estimate the most accurate scores based on the available information for those pollutants.
    * We selected linear regression over mean or mode imputation because these latter methods would have introduced a greater skew to our data. This skew could have negatively impacted our Exploratory Data Analysis (EDA) and the performance of our subsequent models.

In [64]:
# Checking to see how many zeros before cleaning
before_cleaning_data = (data == 0).sum()
print(f"Number of zeros in each feature before cleaning: \n{before_cleaning_data}")

# Checking length of data
print(f"\nTotal length of Data after cleaning: \n{len(data_cleaned)}")

Number of zeros in each feature before cleaning: 
NO2 Mean               8242
NO2 1st Max Value      8274
NO2 1st Max Hour     167581
NO2 AQI               12626
O3 Mean                 596
O3 1st Max Value        596
O3 1st Max Hour       88420
O3 AQI                  596
SO2 Mean             145698
SO2 1st Max Value    152970
SO2 1st Max Hour     158409
SO2 AQI              191184
CO Mean               57654
CO 1st Max Value      58104
CO 1st Max Hour      542747
CO AQI                34134
dtype: int64

Total length of Data after cleaning: 
874422


In [65]:
# Elimiinating any negative or zero values from the data set
data_cleaned = data[(data > 0).all(axis=1)].copy()

#Checking to see hoa many zeros after cleaning
after_cleaning_data = (data_cleaned == 0).sum()
print(f"\nNumber of zeros in each feature after cleaning: \n{after_cleaning_data}")

# Checking length of data
print(f"\nTotal length of Data after cleaning: \n{len(data_cleaned)}")


Number of zeros in each feature after cleaning: 
NO2 Mean             0
NO2 1st Max Value    0
NO2 1st Max Hour     0
NO2 AQI              0
O3 Mean              0
O3 1st Max Value     0
O3 1st Max Hour      0
O3 AQI               0
SO2 Mean             0
SO2 1st Max Value    0
SO2 1st Max Hour     0
SO2 AQI              0
CO Mean              0
CO 1st Max Value     0
CO 1st Max Hour      0
CO AQI               0
dtype: int64

Total length of Data after cleaning: 
874422


In [66]:
# Assign the Overall AQI by taking the max across the specified columns
data_cleaned.loc[:, "Overall AQI"] = data_cleaned[['NO2 AQI', 'O3 AQI', 'SO2 AQI', 'CO AQI']].max(axis=1)

In [67]:
# Summary stats
data_cleaned.describe()

,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,Overall AQI
count,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00,874422.00
mean,15.81,30.81,13.24,29.02,0.02,0.04,10.66,36.67,2.34,5.68,11.31,7.89,0.43,0.78,11.55,8.88,43.21
std,10.00,16.15,7.16,15.33,0.01,0.02,3.17,21.33,2.96,8.47,6.07,11.30,0.35,0.73,7.19,8.33,20.29
min,0.00,1.00,1.00,1.00,0.00,0.00,1.00,1.00,0.00,0.10,1.00,0.08,0.00,0.00,1.00,0.00,5.00
25%,8.46,19.00,6.00,18.00,0.02,0.03,9.00,25.00,0.57,1.30,8.00,1.53,0.22,0.36,6.00,4.12,31.00
50%,13.92,30.00,15.00,28.00,0.02,0.04,10.00,33.00,1.31,3.00,11.00,4.00,0.34,0.60,8.00,6.77,39.00
75%,21.17,40.40,20.00,38.00,0.03,0.05,11.00,42.00,2.96,6.60,16.00,9.06,0.54,0.93,20.00,10.51,47.00
max,139.54,267.00,23.00,132.00,0.10,0.14,23.00,218.00,81.25,292.00,23.00,316.24,6.97,19.60,23.00,221.30,316.24


In [69]:
columns_to_check = ['NO2 AQI', 'O3 AQI', 'CO AQI', 'SO2 AQI', 'Overall AQI']

upper_limit = 200

for column in data_cleaned.columns:
    
    Q1 = data_cleaned[column].quantile(0.25)
    Q3 = data_cleaned[column].quantile(0.75)

    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR

    data_cleaned = data_cleaned[(data_cleaned[column] >= lower_bound) & (data_cleaned[column] <= upper_limit)]

data_cleaned.describe()

,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,Overall AQI
count,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00,864463.00
mean,15.83,30.87,13.22,29.09,0.02,0.04,10.74,36.78,2.33,5.68,11.30,7.89,0.43,0.78,11.51,8.89,43.28
std,9.96,16.05,7.16,15.29,0.01,0.02,3.08,21.23,2.94,8.36,6.05,11.24,0.35,0.73,7.19,8.33,20.16
min,0.00,1.00,1.00,1.00,0.00,0.00,6.00,1.00,0.00,0.10,1.00,0.08,0.00,0.00,1.00,0.00,6.00
25%,8.50,19.00,6.00,18.00,0.02,0.03,9.00,25.00,0.57,1.30,8.00,1.53,0.22,0.37,6.00,4.18,31.00
50%,13.96,30.00,14.00,28.00,0.02,0.04,10.00,33.00,1.31,3.00,11.00,4.00,0.34,0.60,8.00,6.77,39.00
75%,21.21,40.70,20.00,38.00,0.03,0.05,11.00,43.00,2.96,6.60,16.00,9.06,0.54,0.94,20.00,10.70,47.69
max,105.50,199.00,23.00,120.00,0.09,0.12,23.00,200.00,56.08,198.00,23.00,175.97,6.97,17.50,23.00,197.09,200.00


# Handling Outliers
* Given that our data comprises real-world measurements collected over a 16-year period across the United States, it inherently contains instances where pollutant levels are exceptionally high due to specific events.
* Consequently, we decided to retain outliers with measurements below 200. This threshold was chosen because levels exceeding 200 are typically considered acutely dangerous and the probability of such extreme, yet potentially real, occurrences is very low.

In [70]:
# Making new csv with only these columns
file_path = os.path.join(os.getcwd(), 'cleaned_dataset.csv')
data_cleaned.to_csv(file_path, index=False)